# Export slices containing metastases

This notebook goes through each step to export images of the metastases based on the segmentation

In [9]:
# import libraries

import os
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
from skimage.measure import label, regionprops
from pathlib import Path
from tqdm import tqdm

In [16]:
def save_slice_as_image(slice_data, output_path):
    plt.imshow(slice_data, cmap='gray')
    plt.axis('off')
    plt.savefig(output_path, bbox_inches='tight', pad_inches=0)
    plt.close()

# turn images 90 degress
def turn_image_90(image):
    return np.rot90(image, 3)

# mirror images
def mirror_image(image):
    return np.fliplr(image)

def process_nifty_mask(nifty_path, mri_path, output_dir, patient_id=None, mri_sequence=None):
    # Load the NIFTI mask file
    img = nib.load(nifty_path)
    mask_data = img.get_fdata()

    mri_data = nib.load(mri_path).get_fdata()
    
    # Iterate through each slice in the 3D mask
    largest_slices = []
    for i in range(mask_data.shape[2]):
        slice_data = mask_data[:, :, i]
        mri_slice_data = mri_data[:, :, i]
        
        # Find all connected components (metastases)
        labeled_slice = label(slice_data)
        regions = regionprops(labeled_slice)
        
        for region in regions:
            # Calculate the area of each metastasis
            area = region.area
            if len(largest_slices) <= region.label:
                #largest_slices.append((area, slice_data, i, region.label))
                largest_slices.append((area, mri_slice_data, i))
            else:
                if area > largest_slices[region.label-1][0]:
                    #largest_slices[region.label-1] = (area, slice_data, i, region.label)
                    largest_slices[region.label-1] = (area, mri_slice_data, i)
    
    # Save the largest slices as image files
    for area, slice_data, slice_idx in largest_slices:
        if area > 30:
            output_path = os.path.join(output_dir, f'{patient_id}_metastasis_{slice_idx}_{mri_sequence}.png')
            #save_slice_as_image(slice_data, output_path)
            save_slice_as_image(mirror_image(turn_image_90(slice_data)), output_path)
            print(f"Saved largest slice for metastasis (slice {slice_idx}) with area {area} to {output_path}")

# Example usage
mask_file_path = '/Users/LennartPhilipp/Desktop/testing_data/derivatives/segmented_AURORA_20240424-122816/sub-01009590/metastasis_unbinarized_floats.nii.gz'
mri_path = "/Users/LennartPhilipp/Desktop/testing_data/derivatives/preprocessed_brainlesion_20240424-110551/sub-01009590/preprocessed/sub-01009590_fla_bet_normalized.nii.gz"
output_directory = '/Users/LennartPhilipp/Desktop/testing_data/derivatives/met_slices'
os.makedirs(output_directory, exist_ok=True)
process_nifty_mask(mask_file_path, mri_path, output_directory)

## create skript to go through all patients

In [17]:
# output: create folder for each patient and save the slices there

# input mri folder: path to all preprocessed patients
# go to each patient and get all 4 mri scans
# input segmentation folder: path to all segmented patients
# go to each patient and get metastatis_unbinarized_floats.nii.gz

path_to_output = Path("/Users/LennartPhilipp/Desktop/testing_data/derivatives/met_slices")
path_to_mri_patients = Path("/Users/LennartPhilipp/Desktop/testing_data/derivatives/preprocessed_brainlesion_20240424-110551")
path_to_segmentation_patients = Path("/Users/LennartPhilipp/Desktop/testing_data/derivatives/segmented_AURORA_20240424-122816")

segmentation_patients = [patient for patient in os.listdir(path_to_segmentation_patients) if os.path.isdir(os.path.join(path_to_segmentation_patients, patient))]

for patient in tqdm(segmentation_patients):
    # go through each patient
    
    print(f"currently working on {patient}")

    # create new folder for output for patient
    path_to_patient_output = Path(path_to_output) / Path(patient)
    os.makedirs(path_to_patient_output, exist_ok=True)

    # IMPORTANT!!!!!
    # adjust path for final run
    path_to_segmentation = Path(path_to_segmentation_patients) / Path(patient) / "metastasis_unbinarized_floats.nii.gz"

    path_to_mri_images = Path(path_to_mri_patients) / Path(patient) / "preprocessed"
    mri_images = [file for file in os.listdir(path_to_mri_images) if ".nii.gz" in file]

    for mri_image in mri_images:
        # go through each mri image for the patient

        mri_sequence = mri_image.split("_")[1]

        path_to_mri = Path(path_to_mri_images) / Path(mri_image)
        process_nifty_mask(path_to_segmentation, path_to_mri, path_to_patient_output, patient, mri_sequence)
    
    print(f"done with patient {patient}")
    

  0%|          | 0/5 [00:00<?, ?it/s]

currently working on sub-01009590
Saved largest slice for metastasis (slice 56) with area 424.0 to /Users/LennartPhilipp/Desktop/testing_data/derivatives/met_slices/sub-01009590/sub-01009590_metastasis_56_t2.png
Saved largest slice for metastasis (slice 56) with area 424.0 to /Users/LennartPhilipp/Desktop/testing_data/derivatives/met_slices/sub-01009590/sub-01009590_metastasis_56_fla.png
Saved largest slice for metastasis (slice 56) with area 424.0 to /Users/LennartPhilipp/Desktop/testing_data/derivatives/met_slices/sub-01009590/sub-01009590_metastasis_56_t1c.png


 20%|██        | 1/5 [00:00<00:01,  2.29it/s]

Saved largest slice for metastasis (slice 56) with area 424.0 to /Users/LennartPhilipp/Desktop/testing_data/derivatives/met_slices/sub-01009590/sub-01009590_metastasis_56_t1.png
done with patient sub-01009590
currently working on sub-01006290
Saved largest slice for metastasis (slice 63) with area 41.0 to /Users/LennartPhilipp/Desktop/testing_data/derivatives/met_slices/sub-01006290/sub-01006290_metastasis_63_t2.png
Saved largest slice for metastasis (slice 65) with area 33.0 to /Users/LennartPhilipp/Desktop/testing_data/derivatives/met_slices/sub-01006290/sub-01006290_metastasis_65_t2.png
Saved largest slice for metastasis (slice 64) with area 39.0 to /Users/LennartPhilipp/Desktop/testing_data/derivatives/met_slices/sub-01006290/sub-01006290_metastasis_64_t2.png
Saved largest slice for metastasis (slice 63) with area 41.0 to /Users/LennartPhilipp/Desktop/testing_data/derivatives/met_slices/sub-01006290/sub-01006290_metastasis_63_t1c.png
Saved largest slice for metastasis (slice 65) wi

 40%|████      | 2/5 [00:01<00:01,  1.71it/s]

...

Saved largest slice for metastasis (slice 120) with area 66.0 to /Users/LennartPhilipp/Desktop/testing_data/derivatives/met_slices/sub-01005097/sub-01005097_metastasis_120_fla.png
Saved largest slice for metastasis (slice 120) with area 66.0 to /Users/LennartPhilipp/Desktop/testing_data/derivatives/met_slices/sub-01005097/sub-01005097_metastasis_120_t2.png
done with patient sub-01005097
currently working on sub-01015961
Saved largest slice for metastasis (slice 60) with area 552.0 to /Users/LennartPhilipp/Desktop/testing_data/derivatives/met_slices/sub-01015961/sub-01015961_metastasis_60_t2.png
Saved largest slice for metastasis (slice 43) with area 69.0 to /Users/LennartPhilipp/Desktop/testing_data/derivatives/met_slices/sub-01015961/sub-01015961_metastasis_43_t2.png
Saved largest slice for metastasis (slice 60) with area 552.0 to /Users/LennartPhilipp/Desktop/testing_data/derivatives/met_slices/sub-01015961/sub-01015961_metastasis_60_t1c.png
Saved largest slice for metastasis (slice 

 80%|████████  | 4/5 [00:02<00:00,  1.84it/s]

Saved largest slice for metastasis (slice 60) with area 552.0 to /Users/LennartPhilipp/Desktop/testing_data/derivatives/met_slices/sub-01015961/sub-01015961_metastasis_60_fla.png
Saved largest slice for metastasis (slice 43) with area 69.0 to /Users/LennartPhilipp/Desktop/testing_data/derivatives/met_slices/sub-01015961/sub-01015961_metastasis_43_fla.png
Saved largest slice for metastasis (slice 60) with area 552.0 to /Users/LennartPhilipp/Desktop/testing_data/derivatives/met_slices/sub-01015961/sub-01015961_metastasis_60_t1.png
Saved largest slice for metastasis (slice 43) with area 69.0 to /Users/LennartPhilipp/Desktop/testing_data/derivatives/met_slices/sub-01015961/sub-01015961_metastasis_43_t1.png
done with patient sub-01015961
currently working on sub-01005630
Saved largest slice for metastasis (slice 41) with area 93.0 to /Users/LennartPhilipp/Desktop/testing_data/derivatives/met_slices/sub-01005630/sub-01005630_metastasis_41_t2.png
Saved largest slice for metastasis (slice 33) 

100%|██████████| 5/5 [00:02<00:00,  1.86it/s]

Saved largest slice for metastasis (slice 41) with area 93.0 to /Users/LennartPhilipp/Desktop/testing_data/derivatives/met_slices/sub-01005630/sub-01005630_metastasis_41_fla.png
Saved largest slice for metastasis (slice 33) with area 43.0 to /Users/LennartPhilipp/Desktop/testing_data/derivatives/met_slices/sub-01005630/sub-01005630_metastasis_33_fla.png
Saved largest slice for metastasis (slice 41) with area 93.0 to /Users/LennartPhilipp/Desktop/testing_data/derivatives/met_slices/sub-01005630/sub-01005630_metastasis_41_t1.png
Saved largest slice for metastasis (slice 33) with area 43.0 to /Users/LennartPhilipp/Desktop/testing_data/derivatives/met_slices/sub-01005630/sub-01005630_metastasis_33_t1.png
done with patient sub-01005630
